# Test interpolation

Tests my fortran interpolation routine

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

from volrender.volrender import fmodule

In [ ]:
from scipy.interpolate import RegularGridInterpolator

## Define test function
It takes the data, generates random points and overplots the interpolations

In [ ]:
def test(x, y, z, vals, n_p=100, fct=None):

    vmax = vals.max()
    vmin = 0.0
    norm = Normalize(vmin=vmin, vmax=vmax)

    xo, yo = np.random.rand(2, n_p)
    yo *= y[-1]
    zo = np.ones(n_p) * 0.5
    points = np.array([xo, yo, zo]).T

    vals_int = fmodule.interpolate(x, y, z, vals, points)

    
    if fct is not None:
        vals_true = fct(*points.T)
        
    f_scipy = RegularGridInterpolator((x,y,z), vals)
    vals_scipy = f_scipy(points)

    cm = plt.cm.RdBu
    c = cm(norm(vals_int))

    iz = 0
    f, ax = plt.subplots(dpi=200)
    ax.set_aspect('equal')
    cc = ax.pcolormesh(x, y, vals[:, :, iz].T, norm=norm, cmap=cm, shading='gouraud')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.colorbar(cc);
    ax.scatter(xo, yo, c=c, norm=norm, ec='w');
    
    if fct is not None:
        idx = vals_scipy.argsort()
        f, ax = plt.subplots(dpi=200)
        ax.plot(np.arange(n_p), vals_true[idx], 'k', lw=3, label='true')
        ax.plot(np.arange(n_p), vals_scipy[idx], '--', label='scipy')
        ax.plot(np.arange(n_p), vals_int[idx], ':', label='fmodule')
        ax.legend()

## Test 1

In [ ]:
nx = 50
ny = 40
nz = 2

x = np.linspace(0.0, 1.0, nx)
y = np.linspace(0.0, 2.0, ny)
z = np.linspace(0.0, 1.0, nz)

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

def f(x,y,z):
    return np.sqrt(x**2 + y**2) * x * np.sin(2*np.pi * y)

vals = f(X, Y, Z)

In [ ]:
test(x, y, z, vals, fct=f)

## Test data 2

In [ ]:
x = np.array([0.0, 1.0])
y = np.array([0.0, 1.0])
z = np.array([0.0, 1.0])

xi = np.array([0.0, 0.5, 1.0])
yi = np.array([0.0, 0.5, 1.0])
zi = np.array([0.0, 0.5, 1.0])

valsc = np.zeros([2, 2, 2])
valsc[:, 1, :] += 1.0
valsc[1, :, :] += 1.0

def f(x, y, z):
    return x + y

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
valsc = f(X, Y, Z)

In [ ]:
test(x, y, z, valsc, fct=f)